____
### This tutorial provides each step to build and train the regression model.

The model specifies hyperfine parameters with using predicted periods from HPC models. (see details about HPC models in Part 3 and 4 of tutorial)

____
Note1: To run this notebook, first click "Open in playgound" tab above. You may need "Chrome" browser and "Gmail account". Then in each cell, press 'Shift + Enter'

Note2: It doesn't need any packages or programs and also doesn't store any files in your device.

In [ ]:
from imports.models import *
from imports.utils import *
import adabound as Adabound

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import glob
import sys
import time
np.set_printoptions(suppress=True)

In [ ]:
from multiprocessing import Pool
POOL_PROCESS = 23
FILE_GEN_INDEX = 2
pool = Pool(processes=POOL_PROCESS)

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.utils import shuffle
import itertools

In [ ]:
# time index dictionary file corresponding target periods (see Supplementary Note 1)
total_indices = np.load('/content/Deep_Learning_CPMG_Analysis/tutorial_data/total_indices_v4_full.npy', allow_pickle=True).item()

# (A, B) pairs which corresponds each target period. (Equation is described in the paper)
AB_lists_dic = np.load('/content/Deep_Learning_CPMG_Analysis/tutorial_data/AB_target_dic_v4_s0.npy', allow_pickle=True).item()
for i in range(1, 16):
    temp = np.load('/content/Deep_Learning_CPMG_Analysis/tutorial_data/AB_target_dic_v4_s{}.npy'.format(i), allow_pickle=True).item()
    AB_lists_dic.update(temp)

In [ ]:
print("Generating a CPMG signal with target spins.")
MAGNETIC_FIELD = 403.553                        # Unit: Gauss
GYRO_MAGNETIC_RATIO = 1.0705*1000               # Unit: Herts
WL_VALUE = MAGNETIC_FIELD*GYRO_MAGNETIC_RATIO*2*np.pi

CUDA_DEVICE = 0
torch.cuda.set_device(device=CUDA_DEVICE)
N_PULSE = 256
PRE_PROCESS = False
PRE_SCALE = 1

test_spins = np.array([
                       [18000, 2800], # similar spin to C1 in the paper --> a target spin in this tutorial
                       [-23000, 8700],   # test spin                      --> a target spin in this tutorial
                       [5500, 7150],    # residual spin
                       [15100, 27000],   # residual spin
                       [32200, 13260],   # residual spin
                       [47600, 23200],   # residual spin
                       [32800, 46420],   # residual spin
                       [-27000, 53000],   # residual spin
                       [-57000, 11140],   # residual spin
                       [-30100, 5820],   # residual spin
                       ])*2*np.pi

time_resolution = 0.004
time_data = np.arange(0, 60, time_resolution)

IMAGE_WIDTH = 10
TIME_RANGE  = 10000

test_data_M = M_list_return(time_data*1e-6, WL_VALUE, test_spins, N_PULSE)
test_data = (1+test_data_M) / 2

Generating a CPMG signal with target spins.


In [ ]:
fig = go.Figure()
fig.update_layout(
    title={
          'text': "Test CPMG Signal (N=256)",
          'y':0.9,
          'x':0.5,
          'xanchor': 'center',
          'yanchor': 'top'
          },
    xaxis_title="us",
    yaxis_title="Px",
    font=dict(
        family="Arial",
        size=18
    )
)
fig.add_trace(
    go.Scatter(x=time_data, y=test_data, name='CPMG signal')
)
fig.show()

In [ ]:
# Assume that HPC models successfully predict high confidence scores for target preriods of two target spins in the above cell.
# Generally, when a single spin exists, HPC model predicts high confidence scores for the range of 2~4 target periods (if evalutation step size=50Hz).

A_lists = np.array(
      [
       [[17900-50, 17900, 17900+50], [-1]],   # Assume this is the prediction result(=predicted target periods of high confidence scores) from HPC model.
                                              # , [-1] means the number of spin is one within the predicted target periods.
       [[-23050, -23050+50], [-1]],
      ]
    , dtype=object)

In [ ]:
IMAGE_WIDTH = 8
TIME_RANGE  = 5000
num_of_summation = 4

B_init  = 1500
B_final = 25000
noise_scale = 0.07

cpu_num_for_multi = 20    # the number of cpu-thread used for generating datasets
batch_for_multi = 64      # the number of batch size per cpu-thread
class_batch = cpu_num_for_multi*batch_for_multi # the number of batch per class

A_target_margin = 25           # the marginal range of the target spins. (Hz)
A_side_margin = 250            # the marginal range of the side spins. (Hz)
A_final_margin = 20            # the marginal range of the final target spins. (Hz)
distance_btw_target_side = 500 # distance_btw_target_side - (A_target_margin+A_side_margin)
                               # = the final distance between target and side
                               # this distance highly affects the loss during the trainnig
                               # because it gets more difficult as the distance gets narrower

A_far_side_margin = 5000       # the marginal range of the far_side spins. (Hz)
side_candi_num = 5             # the number of "how many times" to generate 'AB_side_candidate'

A_num = 1    # this determines how many sections will be divided within the whole target range of A
B_num = 1    # this determines how many sections will be divided within the whole target range of B
class_num = A_num*B_num + 1   # CAUTION: class_num = 'how many classes in a target AB candidates'
image_width = IMAGE_WIDTH

A_hier_margin = 75
A_existing_margin = 150
B_existing_margin = 2500

___

*   Various parameters could be tested for optimal performance. See below variable lists for width, time, B_init.


In [ ]:
width_list = [10]       # This determines a image width of trainig data. (try any values like [5,10,15,20 ....])
time_list = [5000]      # This determines a total time length to be used for trainig data. (try any values like [2000,3500,5000 ....]) and try plotting it as 2D image to see changes.
B_init_list = [2000]    # This determines a minimum B value when generating trainig data. (try any values like [1000,2000, 3500 ....]) and one may see how this affects accuracies.
B_max_list = [15000]     # This determines a maximum B value when generating trainig data. (try any values like [10000,20000,30000 ....])
samespin_zero_scale_list = [0.5]
parameter_list = [[width, total_time, bmax, B_init, samespin_zero_scale] for width, total_time, bmax, B_init, samespin_zero_scale in itertools.product(width_list, time_list, B_max_list, B_init_list, samespin_zero_scale_list)]
total_pred_list = []
for batch_for_multi in [64, 64, 64, 64, 64, 64, 64]:
    class_batch = cpu_num_for_multi*batch_for_multi # the number of batch per class
    for idx, [width, total_time, B_final, B_init, samespin_zero_scale] in enumerate(parameter_list):
        model_lists = np.array([[A_temp_list[0][0], A_temp_list[0][-1], B_init, B_final] for A_temp_list in A_lists])
        image_width = width
        TIME_RANGE = total_time

        print("================================================================================")
        print('image_width:{}, TIME_RANGE:{}, B_init:{}, B_end:{}'.format(width, total_time, B_init, B_final))
        print("================================================================================")

        for model_idx, [A_first, A_end, B_first, B_end] in enumerate(model_lists):
            hier_target_index = A_lists[model_idx][1][0] # hier_index
            print("========================================================================")
            print('A_first:{}, A_end:{}, B_first:{}, B_end:{}, Bat_mul:{}'.format(A_first, A_end, B_first, B_end, batch_for_multi))
            print("========================================================================")
            data_gen_time = time.time()
            A_resol, B_resol = 50, B_end-B_first

            A_idx_list = np.arange(A_first, A_end+A_resol, A_num*A_resol)
            if (B_end-B_first)%B_resol==0:
                B_idx_list = np.arange(B_first, B_first+B_resol, B_num*B_resol)
            else:
                B_idx_list = np.arange(B_first, B_end, B_num*B_resol)
            AB_idx_set = [[A_idx, B_idx] for A_idx, B_idx in itertools.product(A_idx_list, B_idx_list)]

            A_side_num = 8
            A_side_resol = 600
            B_side_min, B_side_max = 1000, 14000
            B_side_gap = 500  # distance between target and side (applied for both side_same and side)
            B_target_gap = 0

            spin_zero_scale = {'same':samespin_zero_scale, 'side':0.50, 'mid':0.05, 'far':0.05}  # setting 'same'=1.0 for hierarchical model

            epochs = 20
            valid_batch = 4096
            valid_mini_batch = 1024

            args = (AB_lists_dic, N_PULSE, A_num, B_num, A_resol, B_resol, A_side_num, A_side_resol, B_side_min,
                        B_side_max, B_target_gap, B_side_gap, A_target_margin, A_side_margin, A_far_side_margin,
                        class_batch, class_num, spin_zero_scale, distance_btw_target_side, side_candi_num)

            for class_idx in range(num_of_summation):
                TPk_AB_candi, _, temp_hier_target_AB_candi  = gen_TPk_AB_candidates(AB_idx_set, True, *args)
                temp_hier_target_AB_candi[:,:,0] = get_marginal_arr(temp_hier_target_AB_candi[:,:,0], A_hier_margin)
                if class_idx==0:
                    total_hier_target_AB_candi = temp_hier_target_AB_candi[:]
                    target_candidates = TPk_AB_candi[0, :, 0, :]
                    side_candidates   = TPk_AB_candi[1, :, 0, :]
                    rest_candidates   = TPk_AB_candi[1, :, 1:, :]
                else:
                    total_hier_target_AB_candi = np.concatenate((total_hier_target_AB_candi, temp_hier_target_AB_candi[:]), axis=1)
                    target_candidates = np.concatenate((target_candidates, TPk_AB_candi[0, :, 0, :]), axis=0)
                    side_candidates   = np.concatenate((side_candidates, TPk_AB_candi[1, :, 0, :]), axis=0)
                    rest_candidates   = np.concatenate((rest_candidates, TPk_AB_candi[1, :, 1:, :]), axis=0)

            hier_indices = return_total_hier_index_list(A_idx_list, cut_threshold=3)
            hier_indices = return_reduced_hier_indices(hier_indices)
            total_class_num = hier_indices[-1][0].__len__() + 1

            total_TPk_AB_candidates = np.zeros((total_class_num, num_of_summation*TPk_AB_candi.shape[1], total_class_num+TPk_AB_candi.shape[2], 2))
            indices = np.random.randint(rest_candidates.shape[0], size=(total_class_num, rest_candidates.shape[0]))
            total_TPk_AB_candidates[:, :, (total_class_num-1):-2, :] = rest_candidates[indices]
            indices = np.random.randint(side_candidates.shape[0], size=(total_class_num, side_candidates.shape[0], 2))
            total_TPk_AB_candidates[:, :, -2:, :] = side_candidates[indices]

            final_TPk_AB_candidates = total_TPk_AB_candidates[:1]
            for class_idx, hier_index in enumerate(hier_indices):
                temp_batch = total_TPk_AB_candidates.shape[1] // len(hier_index)
                for idx2, index in enumerate(hier_index):
                    temp = np.swapaxes(total_hier_target_AB_candi[index], 0, 1)
                    if idx2 < (len(hier_index)-1):
                        temp_idx = np.random.randint(total_hier_target_AB_candi.shape[1], size=(temp_batch))
                        total_TPk_AB_candidates[class_idx+1, (idx2)*temp_batch:(idx2+1)*temp_batch, :len(index)] = temp[temp_idx]
                    else:
                        residual_batch = total_TPk_AB_candidates[class_idx+1, (idx2)*temp_batch:, :len(index)].shape[0]
                        temp_idx = np.random.randint(total_hier_target_AB_candi.shape[1], size=(residual_batch))
                        total_TPk_AB_candidates[class_idx+1, (idx2)*temp_batch:, :len(index)] = temp[temp_idx]
                final_TPk_AB_candidates = np.concatenate((final_TPk_AB_candidates, total_TPk_AB_candidates[class_idx+1:class_idx+2, :, :]), axis=0)

            median_idx = len(AB_idx_set) // 2
            model_index = get_model_index(total_indices, AB_idx_set[median_idx][0], time_thres_idx=TIME_RANGE-20, image_width=image_width)

            final_TPk_hier_index = len(hier_indices) + hier_target_index + 1
            final_TPk_AB_candidates = final_TPk_AB_candidates[final_TPk_hier_index]
            num_of_target_spins = hier_indices[-1][0].__len__() + hier_target_index + 1
            Y_train_arr = final_TPk_AB_candidates[:, :num_of_target_spins]
            Y_train_arr = Y_train_arr.reshape(-1, num_of_target_spins*2)
            Y_train_arr, scale_lists = Y_train_normalization(Y_train_arr)

            X_train_arr = np.zeros((final_TPk_AB_candidates.shape[0], model_index.shape[0], 2*image_width+1))
            mini_batch = X_train_arr.shape[0] // cpu_num_for_multi
            for idx1 in range(cpu_num_for_multi):
                AB_lists_batch = final_TPk_AB_candidates[idx1*mini_batch:(idx1+1)*mini_batch]
                globals()["pool_{}".format(idx1)] = pool.apply_async(gen_M_arr_batch, [AB_lists_batch, model_index, time_data[:TIME_RANGE],
                                                                                        WL_VALUE, N_PULSE, PRE_PROCESS, PRE_SCALE,
                                                                                        noise_scale])

            for idx2 in range(cpu_num_for_multi):
                X_train_arr[idx2*mini_batch:(idx2+1)*mini_batch] = globals()["pool_{}".format(idx2)].get(timeout=None)
            print("class_idx:", class_idx, end=' ')
            print("Time consumed(s): ", round(time.time() - data_gen_time, 2))
            X_train_arr = X_train_arr.reshape(-1, model_index.flatten().shape[0])

            X_train_arr, Y_train_arr = shuffle(X_train_arr, Y_train_arr)

            model = Regression_model(X_train_arr.shape[-1], Y_train_arr.shape[-1]).cuda()
            try:
                model(torch.Tensor(X_train_arr[:16]).cuda())
            except:
                raise NameError("The input shape should be revised")
            total_parameter = sum(p.numel() for p in model.parameters())
            print('total_parameter: ', total_parameter / 1000000, 'M')

            MODEL_DIR = '/test_regression/'
            try:os.mkdir(MODEL_DIR)
            except:pass
            MODEL_PATH = MODEL_DIR + 'reg_model'

            mini_batch_list = [2048]
            learning_rate_list = [5e-6]
            op_list = [['Adabound', [30,15,7,1]]]
            criterion = nn.MSELoss().cuda()

            hyperparameter_set = [[mini_batch, learning_rate, selected_optim_name] for mini_batch, learning_rate, selected_optim_name in itertools.product(mini_batch_list, learning_rate_list, op_list)]
            print("==================== A_idx: {}, B_idx: {} ======================".format(A_first, B_first))

            total_loss, total_val_loss, total_acc, trained_model = train(MODEL_PATH, N_PULSE, X_train_arr, Y_train_arr,
                                                                         model, hyperparameter_set, criterion, epochs, valid_batch, valid_mini_batch, is_regression=True)

            model.load_state_dict(torch.load(trained_model[0][0]))
            model.eval()
            print("Model loaded as evalutation mode. Model path:", trained_model[0][0])

            test_data_test = test_data[model_index.flatten()]
            test_data_test = 1-(2*test_data_test - 1)
            test_data_test = test_data_test.reshape(1, -1)
            test_data_test = torch.Tensor(test_data_test).cuda()
            pred = model(test_data_test)
            pred = pred.detach().cpu().numpy()

            reversed_pred = reverse_normalization(pred.flatten(), scale_lists)
            total_pred_list.append(reversed_pred)
            print('Target Value: ', test_spins[model_idx]/2/np.pi)
            print('Prediction Value: ', reversed_pred)
            print('\n This predicted value is used as the initial guess value for the fine-tuning step.')

image_width:10, TIME_RANGE:5000, B_init:2000, B_end:15000
A_first:17850, A_end:17950, B_first:2000, B_end:15000, Bat_mul:64
class_idx: 0 Time consumed(s):  24.59
total_parameter:  7.60781 M
==================== A_idx: 17850, B_idx: 2000 ======================
train_batch:  11264 valid_batch:  4096


 Training Start:  Fri Dec  3 03:23:31 2021
 mini_batch: 2048  | learning_rate:  5e-06  | selected_optim_name:  ['Adabound', [30, 15, 7, 1]]  |
Epoch:    1  | Loss = 0.43320 | Val_loss: 0.25644 % | time: 0.367(s) | lr:  5e-06
Epoch:    2  | Loss = 0.17132 | Val_loss: 0.23478 % | time: 0.317(s) | lr:  5e-06
Epoch:    3  | Loss = 0.08540 | Val_loss: 0.20355 % | time: 0.306(s) | lr:  5e-06
Epoch:    4  | Loss = 0.06217 | Val_loss: 0.16133 % | time: 0.28(s) | lr:  5e-06
Epoch:    5  | Loss = 0.05612 | Val_loss: 0.13853 % | time: 0.285(s) | lr:  5e-06
Epoch:    6  | Loss = 0.04573 | Val_loss: 0.11495 % | time: 0.284(s) | lr:  5e-06
Epoch:    7  | Loss = 0.03790 | Val_loss: 0.08802 % | time: 0.291

In [ ]:
total_pred_list = np.array(total_pred_list)
first_spin = (total_pred_list[0] + total_pred_list[2] + total_pred_list[4]) / 3
second_spin = (total_pred_list[1] + total_pred_list[3] + total_pred_list[5]) / 3

print("Averaged Prediction Value of 1st target spin: ",
      "A: {0} (target A: {1})".format(round(first_spin[0],2), round(test_spins[0,0]/2/np.pi,2)),
      "B: {0} (target B: {1})".format(round(first_spin[1],2), round(test_spins[0,1]/2/np.pi,2)))
print("Averaged Prediction Value of 2nd target spin: ",
      "A: {0} (target A: {1})".format(round(second_spin[0],2), round(test_spins[1,0]/2/np.pi,2)),
      "B: {0} (target B: {1})".format(round(second_spin[1],2), round(test_spins[1,1]/2/np.pi,2)))

Averaged Prediction Value of 1st target spin:  A: 18000.99 (target A: 18000.0) B: 3756.08 (target B: 2800.0)
Averaged Prediction Value of 2nd target spin:  A: -23006.94 (target A: -23000.0) B: 7886.6 (target B: 8700.0)


### The results are used as inital guess values for Particle Swarm Optimization (PSO) tuning method.